In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/MRP/creditcard.csv')
temp = pd.read_csv('/content/drive/My Drive/Colab Notebooks/MRP/creditcard.csv')

In [ ]:
data.groupby('Class')['Class'].count()

Class
0    284315
1       492
Name: Class, dtype: int64

In [ ]:
legitimate_mask = data['Class'] == 0
anomaly_mask = data['Class'] != 0

data.drop ('Class', axis = 1, inplace = True)

data_legitimate = data[legitimate_mask]
data_anomaly = data[anomaly_mask]

print(f"Legitimate Count: {len(data_legitimate)}")
print(f"Anomaly Count: {len(data_anomaly)}")

Legitimate Count: 284315
Anomaly Count: 492


In [ ]:
X_legitimate = data_legitimate.values
X_anamoly = data_anomaly.values

In [ ]:
from sklearn.model_selection import train_test_split

X_legitimate_train, X_legitimate_test = train_test_split(X_legitimate, test_size = 0.25, random_state=101)

In [ ]:
X = temp.drop('Class', axis = 1).values
y = temp['Class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=101)

In [ ]:
print(f"legitimate train Count: {len(X_legitimate_train)}")
print(f"legitimate test Count: {len(X_legitimate_test)}")

legitimate train Count: 213236
legitimate test Count: 71079


In [ ]:
import tensorflow as tf
from keras import models, layers
from keras.layers import Dense
from keras.models import Sequential

Using TensorFlow backend.


In [ ]:
model = Sequential()
model.add(Dense(25, input_dim = X_legitimate_train.shape[1], activation = 'relu'))
model.add(Dense(3, activation = 'relu'))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(X_legitimate_train.shape[1]))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_legitimate_train, X_legitimate_train, batch_size = 10240, verbose = 1, epochs = 200)

Epoch 1/200
213236/213236 [==============================] - 0s 2us/step - loss: 366298750.8853
Epoch 2/200
213236/213236 [==============================] - 0s 1us/step - loss: 338784537.9402
Epoch 3/200
213236/213236 [==============================] - 0s 1us/step - loss: 282393884.5322
Epoch 4/200
213236/213236 [==============================] - 0s 1us/step - loss: 196717309.2604
Epoch 5/200
213236/213236 [==============================] - 0s 1us/step - loss: 98215352.5679
Epoch 6/200
213236/213236 [==============================] - 0s 1us/step - loss: 20048815.1726
Epoch 7/200
213236/213236 [==============================] - 0s 1us/step - loss: 416777.5690
Epoch 8/200
213236/213236 [==============================] - 0s 1us/step - loss: 218223.9042
Epoch 9/200
213236/213236 [==============================] - 0s 1us/step - loss: 11247.4034
Epoch 10/200
213236/213236 [==============================] - 0s 1us/step - loss: 4719.3262
Epoch 11/200
213236/213236 [============================

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 25)                775       
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 78        
_________________________________________________________________
dense_7 (Dense)              (None, 25)                100       
_________________________________________________________________
dense_8 (Dense)              (None, 30)                780       
Total params: 1,733
Trainable params: 1,733
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict_classes(X_test)
predictions = np.where(predictions > 0, 1, 0)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71077
           1       0.00      0.00      0.00       125

    accuracy                           1.00     71202
   macro avg       0.50      0.50      0.50     71202
weighted avg       1.00      1.00      1.00     71202



In [ ]:
confusion_matrix(y_test,predictions)

array([[71046,    31],
       [  125,     0]])

In [ ]:
from sklearn import metrics

pred = model.predict(X_legitimate_test)
score1 = np.sqrt(metrics.mean_squared_error(pred, X_legitimate_test))
print(f"In Sample legitimate Score (RMSE): {score1}".format(score1))

pred = model.predict(X_legitimate)
score2 = np.sqrt(metrics.mean_squared_error(pred, X_legitimate))
print(f"Out of Sample legitimate Score (RMSE): {score2}")

pred = model.predict(X_anamoly)
score3 = np.sqrt(metrics.mean_squared_error(pred, X_anamoly))
print(f"Anomaly Score (RMSE): {score3}".format(score1))

In Sample legitimate Score (RMSE): 8.666338493987373
Out of Sample legitimate Score (RMSE): 8.668498826207781
Anomaly Score (RMSE): 10.083318076323843
